## Final Search Scraper

Author: Miraya Gupta\
Date: 17/04

In [ ]:
import requests 
import selenium
from seleniumbase import Driver
from selenium import webdriver
from selenium.webdriver.common.by import By # contains operators for the type of search we want to do
import time
from seleniumbase import BaseCase
from random import randint
#from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import html
#import re
import numpy as np
from datetime import datetime
import os.path
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import json
import csv

## 1. Dividing Search Terms into Chunks

In [1]:
#open csv of search terms 
#read into pandas
df = pd.read_csv()

In [ ]:
#create 2 chunks randomly 
first_half = df.sample(frac = 0.5)
second_half = df.drop(first_half.index)

In [ ]:
#create 4 chunks from the 2 chunks
first_quarter = first_half.sample(frac = 0.5)
second_quarter = first_half.drop(first_quarter.index)

third_quarter = second_half.sample(frac = 0.5)
fourth_quarter = second_half.drop(third_quarter.index)

## 2. Scraping TikTok Search Results

In [ ]:
#start with driver instance so we don't have to login multiple times. 
driver = webdriver.Chrome()
driver.get(f'https://www.tiktok.com')
actions = ActionChains(driver)

In [ ]:
def get_50_urls(search_term, no_of_results=10):
    #open link using driver
    driver.get(f'https://www.tiktok.com/search/video?q={search_term}&t={no_of_results}')
    #time.sleep(20)
    #perform scrolling and collect urls
    all_urls = []
    for i in range(6):
        for i in range(30):
            actions.send_keys(Keys.ARROW_DOWN)
        actions.perform()
        time.sleep(0.75)
        videos = driver.find_elements(By.XPATH, "//div[contains(@class, 'css-1soki6-DivItemContainerForSearch e19c29qe10')]")
        [all_urls.append(video.find_element (By. TAG_NAME, "a").get_attribute('href')) for video in videos if video not in all_urls]

    #return search term and list of urls as a tuple
    return search_term, all_urls[:50]

In [ ]:
#function to take a df of keywords and run TikTok search on all of them. Outputs a json file of the search terms as 
# keys and the list of 50 urls as values. 
def run_all_search_terms(search_terms_list):
    result_dict = {}
    for search_term in search_terms_list:
        data = get_50_urls(search_term, no_of_results=10)
        keyword = data[0]
        urls = data[1]
        result_dict[keyword] = urls
    with open('results.json', 'w') as f:
        json.dump(result_dict, f)    